In [1]:
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

In [2]:
server_params = StdioServerParameters(
    command='python',
    args=['-m', 'mcp_this'],
)

async with stdio_client(server_params) as (read, write):  # noqa: SIM117
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await session.list_tools()
        print("Available tools:\n-----------------------\n")
        for tool in tools.tools:
            print(f"- `{tool.name}`")

        dir_tree_result = await session.call_tool(
            'get-directory-tree',
            {'directory': '..'},
        )
        code_info_result = await session.call_tool(
            'extract-code-info',
            {
                'files': '/Users/shanekercheval/repos/mcp-this/src/mcp_this/*.py',
                'types': 'functions,classes',
            },
        )

Available tools:
-----------------------

- `get-directory-tree`
- `find-files`
- `find-text-patterns`
- `extract-file-text`
- `extract-code-info`
- `edit-file`
- `create-file`
- `create-directory`
- `web-scraper`


In [3]:
dir_tree_result

CallToolResult(meta=None, content=[TextContent(type='text', text='..\n├── .github\n│\xa0\xa0 └── workflows\n│\xa0\xa0     └── tests.yaml\n├── .gitignore\n├── .ruff.toml\n├── LICENSE\n├── Makefile\n├── README.md\n├── examples\n│\xa0\xa0 ├── claude_desktop_config_default.json\n│\xa0\xa0 ├── cli.py\n│\xa0\xa0 ├── examples-default-tools.ipynb\n│\xa0\xa0 ├── examples-passing-tools.ipynb\n│\xa0\xa0 ├── mcp_config_cli.json\n│\xa0\xa0 ├── toolset_example__curl.yaml\n│\xa0\xa0 └── top_level_tools_example.yaml\n├── pyproject.toml\n├── src\n│\xa0\xa0 └── mcp_this\n│\xa0\xa0     ├── __init__.py\n│\xa0\xa0     ├── __main__.py\n│\xa0\xa0     ├── configs\n│\xa0\xa0     │\xa0\xa0 └── default.yaml\n│\xa0\xa0     ├── mcp_server.py\n│\xa0\xa0     └── tools.py\n├── temp\n├── tests\n│\xa0\xa0 ├── fixtures\n│\xa0\xa0 │\xa0\xa0 └── test_config.yaml\n│\xa0\xa0 ├── test_command_execution_edge_cases.py\n│\xa0\xa0 ├── test_config_files.py\n│\xa0\xa0 ├── test_config_loading.py\n│\xa0\xa0 ├── test_default_config.p

In [4]:
print(dir_tree_result.content[0].text)

..
├── .github
│   └── workflows
│       └── tests.yaml
├── .gitignore
├── .ruff.toml
├── LICENSE
├── Makefile
├── README.md
├── examples
│   ├── claude_desktop_config_default.json
│   ├── cli.py
│   ├── examples-default-tools.ipynb
│   ├── examples-passing-tools.ipynb
│   ├── mcp_config_cli.json
│   ├── toolset_example__curl.yaml
│   └── top_level_tools_example.yaml
├── pyproject.toml
├── src
│   └── mcp_this
│       ├── __init__.py
│       ├── __main__.py
│       ├── configs
│       │   └── default.yaml
│       ├── mcp_server.py
│       └── tools.py
├── temp
├── tests
│   ├── fixtures
│   │   └── test_config.yaml
│   ├── test_command_execution_edge_cases.py
│   ├── test_config_files.py
│   ├── test_config_loading.py
│   ├── test_default_config.py
│   ├── test_main.py
│   ├── test_mcp_server.py
│   └── test_tool_registration.py
└── uv.lock

10 directories, 28 files



In [5]:
print(code_info_result.content[0].text)

Error executing command: /bin/sh: -c: line 1: syntax error: unexpected end of file

